In [1]:
pip install imbalanced-learn scikit-learn opencv-python numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cv2
import os
from collections import Counter

# Step 1: Load Your Image Dataset
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Use grayscale for simplicity
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize to fixed dimensions
            images.append(img)
            labels.append(label)
    return images, labels


In [3]:
fractured_folder = "train/train_fractured"
unfractured_folder = "train/train_notfractured"
test_fractured_folder="test/fractured"
test_unfractured_folder="test/not_fractured"

In [4]:
fractured_images, fractured_labels = load_images_from_folder(fractured_folder, label=0)  # 0 for fractured
unfractured_images, unfractured_labels = load_images_from_folder(unfractured_folder, label=1)  # 1 for unfractured
test_fractured_images, test_fractured_labels = load_images_from_folder(test_fractured_folder, label=0)
test_unfractured_images, test_unfractured_labels = load_images_from_folder(test_unfractured_folder, label=1)

In [5]:
X_train= np.array(fractured_images + unfractured_images)
y_train = np.array(fractured_labels + unfractured_labels)

X_test = np.array(test_fractured_images + test_unfractured_images)
y_test = np.array(test_fractured_labels + test_unfractured_labels)

In [6]:
X_train=X_train.reshape(len(X_train),-1)


In [7]:
print(f"Original dataset distribution: {Counter(y_train)}")

Original dataset distribution: Counter({np.int64(0): 4603, np.int64(1): 280})


In [8]:
smote = SMOTE(sampling_strategy='minority', random_state=42)



In [9]:
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [10]:
print(f"Resampled training dataset distribution: {Counter(y_train_resampled)}")


Resampled training dataset distribution: Counter({np.int64(0): 4603, np.int64(1): 4603})


In [11]:
X_train_resampled = X_train_resampled.reshape(len(X_train_resampled), 128, 128)


In [12]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
# Normalize the image pixel values
X_train_resampled = X_train_resampled / 255.0
X_test = X_test.reshape(len(X_test),128,128) / 255.0

y_train_resampled = np.array(y_train_resampled).reshape(-1)  # Flatten to 1D
y_test = np.array(y_test).reshape(-1)  # Flatten to 1D

# Convert labels to categorical format for multi-class classification
y_train_resampled = to_categorical(y_train_resampled, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Reshape the resampled training data back into image format (128x128x1)
X_train_resampled = X_train_resampled.reshape((-1, 128, 128, 1))
X_test = X_test.reshape((-1, 128, 128, 1))


In [15]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train_resampled)

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: fractured and unfractured
])


c:\Users\newma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Step 3: Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Train the Model
history = model.fit(
    X_train_resampled, y_train_resampled,
    validation_split=0.2,  # Use 20% of the training data for validation
    epochs=25,             # Number of training epochs
    batch_size=32,         # Number of samples per batch
    verbose=1
)

Epoch 1/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 66s 278ms/step - accuracy: 0.7819 - loss: 0.4553 - val_accuracy: 0.8534 - val_loss: 0.3220
Epoch 2/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 65s 281ms/step - accuracy: 0.9459 - loss: 0.1344 - val_accuracy: 0.9609 - val_loss: 0.0947
Epoch 3/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 62s 270ms/step - accuracy: 0.9805 - loss: 0.0549 - val_accuracy: 0.9929 - val_loss: 0.0259
Epoch 4/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 62s 270ms/step - accuracy: 0.9852 - loss: 0.0403 - val_accuracy: 0.9962 - val_loss: 0.0192
Epoch 5/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 65s 282ms/step - accuracy: 0.9920 - loss: 0.0189 - val_accuracy: 0.9978 - val_loss: 0.0050
Epoch 6/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 61s 265ms/step - accuracy: 0.9930 - loss: 0.0216 - val_accuracy: 0.9973 - val_loss: 0.0162
Epoch 7/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 62s 268ms/step - accuracy: 0.9956 - loss: 0.0145 - val_accuracy: 0.9984 - val_loss: 0.0062
Epoch 8/25
231/231 ━━━━━━━━━━━━━━━━━━━━ 58s 253ms/step - accuracy: 0.9955 - loss: 0

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Make Predictions
y_pred_probs = model.predict(X_test)  # Get predicted probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoded test labels to class labels

# Step 2: Generate Classification Report
report = classification_report(y_true, y_pred, target_names=['Fractured', 'Unfractured'])
print("Classification Report:\n")
print(report)

# Step 3 (Optional): Display Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n")
print(conf_matrix)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
Classification Report:

              precision    recall  f1-score   support

   Fractured       0.63      0.98      0.77       238
 Unfractured       0.97      0.49      0.65       268

    accuracy                           0.72       506
   macro avg       0.80      0.74      0.71       506
weighted avg       0.81      0.72      0.71       506

Confusion Matrix:

[[234   4]
 [136 132]]
